# Finetune LLama-3-8b

In [ ]:
from unsloth import FastLanguageModel 
import torch
from transformers import TextStreamer

## Prompt

In [ ]:
# my_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

In [ ]:
my_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<|eot_id|><|start_header_id|>instruction<|end_header_id|>
{}<|eot_id|><|start_header_id|>input<|end_header_id|>
{}<|eot_id|><|start_header_id|>response<|end_header_id|>
{}<|eot_id|>
"""

max_seq_length = 8192 
dtype = None 
load_in_4bit = True 

fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",    
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",      
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct", 
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
] 

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf 
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "embed_tokens", "lm_head"],
    lora_alpha=16,
    lora_dropout=0, # Supports any, but = 0 is optimized
    bias="none", # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth", # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None, # And LoftQ
)

### Data Prep

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
EOS_TOKEN

In [ ]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = my_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True,)

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


import os
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False, # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        num_train_epochs=5, # Choose 1 to 5 passes
        max_grad_norm=0.3,
        per_device_train_batch_size=8,
        # per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        warmup_steps=5,
        # max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=2000,
        save_steps=2000,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="./Results",
        report_to="tensorboard"
    ),
)

In [ ]:
# trainer_stats = trainer.train("checkpoint-9500") 
trainer_stats = trainer.train()

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# my_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer([my_prompt.format(
    "Help me plan my day", # instruction
    "", # input
    "", # output - leave this blank for generation!
    )], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=1024, use_cache=True)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("llama3_8b_instruct_lora_5epo") # Local saving
tokenizer.save_pretrained("llama3_8b_instruct_lora_5epo")
# model.push_to_hub("trantuan225/my_lora_llama3_model", token="") # Online saving
# tokenizer.push_to_hub("trantuan225/my_lora_llama3_model", token="") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

### Inference after save as lora adapter - `lora_llama3`

In [ ]:
max_seq_length = 8000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


my_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<|eot_id|>


<|start_header_id|>instruction<|end_header_id|>{}<|eot_id|>


<|start_header_id|>input<|end_header_id|>{}<|eot_id|>


<|start_header_id|>response<|end_header_id|>{}<|eot_id|>
"""


if True:
    from transformers import TextStreamer
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        # model_name="llama3_8b_instruct_lora", # YOUR MODEL YOU USED FOR TRAINING
        model_name="llama3_8b_instruct_lora_5epo_merged16bit",
        # model_name="llama3_8b_instruct_lora_5epo", 
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# my_prompt = You MUST copy from above!

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
inputs = tokenizer([my_prompt.format(
    "What is the famous food in Vietnam?", # instruction
    "", # input
    "", # output - leave this blank for generation!
    )], return_tensors="pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=1024, use_cache=True)

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
# if True: model.save_pretrained_merged("llama3_8b_instruct_lora_5epo_merged16bit", tokenizer, save_method="merged_16bit",)
# if True: model.push_to_hub_merged(
#     "trantuan225/llama3_8b_instruct_lora_5epo_merged16bit", 
#     tokenizer, 
#     save_method="merged_16bit",
#     token="") # <=== Chưa cần dùng bây giờ 

# Merge to 4bit
# if True: model.save_pretrained_merged("llama3_8b_instruct_lora_5epo_merged4bit", tokenizer, save_method="merged_4bit",)
# if True: model.push_to_hub_merged("trantuan225/llama3_8b_instruct_lora_20epo_merged16bit_merged4bit", tokenizer, save_method = "merged_4bit", token = "")

# # Just LoRA adapters
# if True: model.save_pretrained_merged("llama3_8b_instruct_lora_mergedlora", tokenizer, save_method = "lora",)
# if False: model.push_to_hub_merged("trantuan225/llama3_8b_instruct_lora_mergedlora", tokenizer, save_method = "lora", token = "")

### Inference after save float16 for VLLM - use `merged_16bit`

In [ ]:
from unsloth import FastLanguageModel 
import torch
from transformers import TextStreamer


max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


my_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<|eot_id|>


<|start_header_id|>instruction<|end_header_id|>{}<|eot_id|>


<|start_header_id|>input<|end_header_id|>{}<|eot_id|>


<|start_header_id|>response<|end_header_id|>{}<|eot_id|>
"""

if True:
    from transformers import TextStreamer
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        # model_name="llama3_8b_instruct_lora_5epo", # LORA 
        model_name="llama3_8b_instruct_lora_5epo_merged16bit", # merged-16b-it 
        # model_name="llama3_8b_instruct_lora_5epo_merged16bit_gguf", # GGUF format 
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# my_prompt = You MUST copy from above!

inputs = tokenizer([my_prompt.format(
    "What is famous food in Vietnam ?", # instruction
    "", # input
    "", # output - leave this blank for generation!
    )], return_tensors="pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128, use_cache=True)

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# # Save to 8bit Q8_0
# if False: model.save_pretrained_gguf("lora_llama3_finetuned", tokenizer,)
# if False: model.push_to_hub_gguf("hf/lora_llama3_finetuned", tokenizer, token = "")

# Save to 16bit GGUF
# if True: model.save_pretrained_gguf("llama3_8b_instruct_lora_5epo_merged16bit_gguf", tokenizer, quantization_method = "f16") # ==> Đã xong
# if True: model.push_to_hub_gguf("trantuan225/llama3_8b_instruct_lora_merged16bit_gguf", tokenizer, quantization_method = "f16", token="")

# # Save to q4_k_m GGUF
# if True: model.save_pretrained_gguf("llama3_8b_instruct_lora_5epo_merged4bit_q4_k_m_gguf", tokenizer, quantization_method = "q4_k_m")
# if False: model.push_to_hub_gguf("hf/lora_llama3_finetuned", tokenizer, quantization_method = "q4_k_m", token = "")

### Evaluation using `llm-evaluation-harness`

In [ ]:
import os
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"

In [9]:
# Local eval `llama3_8b_instruct_lora_5epo_merged16bit_gguf`
%lm_eval --model hf --model_args pretrained=/rhome/f111169109/tools-for-llms-finetune-and-inference/notebooks/llama3/llama3_8b_instruct_lora_5epo_merged16bit_gguf --tasks lambada_openai,hellaswag,piqa,arc_easy,arc_challenge,winogrande,openbookqa --device cuda --batch_size auto 
# Local eval `llama3_8b_instruct_lora_5epo_merged16bit`
%lm_eval --model hf --model_args pretrained=/rhome/f111169109/tools-for-llms-finetune-and-inference/notebooks/llama3/llama3_8b_instruct_lora_5epo_merged16bit --tasks lambada_openai,hellaswag,piqa,arc_easy,arc_challenge,winogrande,openbookqa --device cuda --batch_size auto 
# Local eval `llama3_8b_instruct_lora_5epo`
%lm_eval --model hf --model_args pretrained=/rhome/f111169109/tools-for-llms-finetune-and-inference/notebooks/llama3/llama3_8b_instruct_lora_5epo --tasks lambada_openai,hellaswag,piqa,arc_easy,arc_challenge,winogrande,openbookqa --device cuda --batch_size auto 

### Release VRAM

In [ ]:
# nvidia-smi | grep 'python' | awk '{ print $5 }' | xargs -n1 kill -9